# Test Database connection string 

In [43]:
from pymongo import MongoClient

# Replace the uri string with your MongoDB deployment's connection string.

USERNAME = 'tradingbot_admin'
PASSWORD = 'tradingbot_pass'
CLUSTERNAME = 'tb-cluster-0'

connection_uri = "mongodb+srv://{username}:{password}@{cluster_name}.ztadynx.mongodb.net/?retryWrites=true&w=majority"

uri = connection_uri.format(
    username = USERNAME,
    password = PASSWORD,
    cluster_name = CLUSTERNAME 
)

# client = MongoClient(connection_url)
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


# Retrieve data

In [7]:
# set database and collection code goes here
db = client.sample_guides
coll = db.planets

# find code goes here
cursor = coll.find({"hasRings": False})

# iterate code goes here
for doc in cursor:
    print(doc)


{'_id': ObjectId('621ff30d2a3e781873fcb65c'), 'name': 'Mercury', 'orderFromSun': 1, 'hasRings': False, 'mainAtmosphere': [], 'surfaceTemperatureC': {'min': -173, 'max': 427, 'mean': 67}}
{'_id': ObjectId('621ff30d2a3e781873fcb65e'), 'name': 'Mars', 'orderFromSun': 4, 'hasRings': False, 'mainAtmosphere': ['CO2', 'Ar', 'N'], 'surfaceTemperatureC': {'min': -143, 'max': 35, 'mean': -63}}
{'_id': ObjectId('621ff30d2a3e781873fcb661'), 'name': 'Earth', 'orderFromSun': 3, 'hasRings': False, 'mainAtmosphere': ['N', 'O2', 'Ar'], 'surfaceTemperatureC': {'min': -89.2, 'max': 56.7, 'mean': 14}}
{'_id': ObjectId('621ff30d2a3e781873fcb662'), 'name': 'Venus', 'orderFromSun': 2, 'hasRings': False, 'mainAtmosphere': ['CO2', 'N'], 'surfaceTemperatureC': {'min': None, 'max': None, 'mean': 464}}


# Query database
Try one query at a time

In [8]:
# Set database here:
db = client.sample_guides
coll = db.planets

######-------------- Queries ----------------------------------------

## Query using a single criteria
# cursor = coll.find({"hasRings": False})

## Query with multiple criteria (implicit AND)
# cursor = coll.find({"hasRings": False, "mainAtmosphere": "Ar"})

## Query using the 'less than' -- operator $lt (<)
# cursor = coll.find({"surfaceTemperatureC.mean": {"$lt":15}})

## Compound query (AND)
# cursor = coll.find({"surfaceTemperatureC.mean": {"$lt": 15}, "surfaceTemperatureC.min": {"$gt": -100}})

## Compound query on the same field ---> explicit AND
# cursor = coll.find({"$and": [{"orderFromSun": {"$gt": 2}}, {"orderFromSun": {"$lt": 5}}]})

## not the same as when ommiting the $and operator
# cursor = coll.find({"orderFromSun": {"$gt": 2}}, {"orderFromSun": {"$lt": 5}})

## Queries with mutually exclusive criteria -- OR operator
cursor = coll.find(
    {
        "$or": [
            {"orderFromSun": {"$gt": 7}},
            {"orderFromSun": {"$lt": 2}},
        ]
    }
)


######---------------------------------------------------------------

# iterate code goes here
for doc in cursor:
    print(doc)


# # Close the connection to MongoDB when you're done.
# client.close()

{'_id': ObjectId('621ff30d2a3e781873fcb65c'), 'name': 'Mercury', 'orderFromSun': 1, 'hasRings': False, 'mainAtmosphere': [], 'surfaceTemperatureC': {'min': -173, 'max': 427, 'mean': 67}}
{'_id': ObjectId('621ff30d2a3e781873fcb65f'), 'name': 'Neptune', 'orderFromSun': 8, 'hasRings': True, 'mainAtmosphere': ['H2', 'He', 'CH4'], 'surfaceTemperatureC': {'min': None, 'max': None, 'mean': -201}}


# insert - update - delete data

In [9]:
# select database & create collection if non existent:
db = client.sample_guides
coll = db.comets

coll.drop()

######--------------------------------- insert data ----------------------------------------------

docs = [
    {"name": "Halley's Comet", "officialName": "1P/Halley", "orbitalPeriod": 75, "radius": 3.4175, "mass": 2.2e14},
    {"name": "Wild2", "officialName": "81P/Wild", "orbitalPeriod": 6.41, "radius": 1.5534, "mass": 2.3e13},
    {"name": "Comet Hyakutake", "officialName": "C/1996 B2", "orbitalPeriod": 17000, "radius": 0.77671, "mass": 8.8e12},
    ]

result = coll.insert_many(docs)

# verification display the results of your operation
print(result.inserted_ids)


######--------------------------------- update data ----------------------------------------------

# multiply ($mul) radius field by convertion factor: 1 mile = 1.60934 km
doc = {"$mul": {"radius": 1.60934}}
result = coll.update_many({}, doc)

# display the results of your operation
print("Number of documents updated: ", result.modified_count)


######--------------------------------- delete data ----------------------------------------------

# using a query filter to delete documents where their orbitalPeriod is greater than 5 and less than 85
doc = {
    "orbitalPeriod": {
        "$gt": 5,
        "$lt": 85
    }
}

result = coll.delete_many(doc)

# amount deleted code goes here
print("Number of documents deleted: ", result.deleted_count)

######--------------------------------------------------------------------------------------------

[ObjectId('65d8bb1525f5526235a3a75e'), ObjectId('65d8bb1525f5526235a3a75f'), ObjectId('65d8bb1525f5526235a3a760')]
Number of documents updated:  3
Number of documents deleted:  2


# Upload serialized model

In [15]:
ls "../models"

 Volume in drive C is OS
 Volume Serial Number is 96E8-C330

 Directory of C:\Ramiro\Data Science\0_DataScientest\04_MLOps_project\TradingBotApp_mlops\models

23/02/2024  15:38    <DIR>          .
23/02/2024  15:51    <DIR>          ..
23/02/2024  15:38                 0 .gitkeep
23/02/2024  15:38         3.882.372 model_test.joblib
23/02/2024  15:38                54 model_test_params.json
               3 File(s)      3.882.426 bytes
               2 Dir(s)  140.584.251.392 bytes free


In [62]:
from joblib import dump, load
import bson.binary

# Path to the model file
models_path = "../models/"
model_name = "model_test"
model_filepath = models_path + model_name + ".joblib"

# Load the model
model = load(model_filepath)

# Read the serialized model bytes
with open(model_filepath, 'rb') as f:
    model_bytes = bson.binary.Binary(f.read())

# Access the database and collection
db = client['trading_bot']
collection = db['models']

# Data to insert
doc = {"name": model_name, "model": model_bytes}

# Insert the model bytes into the collection
collection.insert_one(doc)


InsertOneResult(ObjectId('65d8cef925f5526235a3a76d'), acknowledged=True)

### Retrieve the model from cluster

In [63]:
from joblib import dump, load
import bson.binary

# Load the model from MongoDB
db = client['trading_bot']
collection = db['models']
doc = collection.find_one({"name": model_name})

if doc:
    model_bytes = doc["model"]

    # Create a BytesIO object to store the binary data
    model_buffer = BytesIO(model_bytes)

    # Load the model from the BytesIO object - to correctly handle utf-encoding
    model = load(model_buffer)

    # Now you can use the loaded model as needed
    print("Model loaded successfully:", model)
else:
    print("Model not found in the database.")


Model loaded successfully: KNeighborsClassifier(n_neighbors=32)


# Close connection
always

In [64]:
# Close the connection to MongoDB when you're done.
client.close()

In [65]:
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Cannot use MongoClient after close
